#### Libraries

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from requests_html import HTMLSession

from tqdm.notebook import tqdm

In [2]:
#Read excel files and create the dfs
hefzi_df = pd.read_excel('../Data/iCHOv1_Reconstruction.xlsx', header = 1)
fouladiha_df = pd.read_excel('../Data/10529_2020_3021_MOESM1_ESM.xlsx', 'Supplementary Table 10', header = 1)
yeo_df = pd.read_excel('../Data/Supplementary Data.xlsx', 'Data S2')

In [3]:
#Standarization of the columns names
fouladiha_df.rename(columns = {'Abbreviation':'Reaction', 'Description':'Reaction Name', 'Reaction':'Reaction Formula'}, inplace = True)
yeo_df.rename(columns = {'Rxn':'Reaction', 'Subsystem (iCHO1766)':'Subsystem'}, inplace = True)

# Addition of tag columns
hefzi_df.insert(loc=0, column='hef', value='X')
hefzi_df.insert(loc=1, column='fou', value=np.nan)
hefzi_df.insert(loc=2, column='yeo', value=np.nan)

fouladiha_df.insert(loc=0, column='hef', value=np.nan)
fouladiha_df.insert(loc=1, column='fou', value='X')
fouladiha_df.insert(loc=2, column='yeo', value=np.nan)

yeo_df.insert(loc=0, column='hef', value=np.nan)
yeo_df.insert(loc=1, column='fou', value=np.nan)
yeo_df.insert(loc=2, column='yeo', value='X')


hefzi_df

,hef,fou,yeo,Reaction,Reaction Name,Subsystem,Reaction Formula,GPR,Curation Notes,References
0,X,NaN,NaN,COKECBESr,Carboxylesterase (cocaine) (er),ALKALOID SYNTHESIS,coke[r] + h2o[r] -> bz[r] + egme[r] + h[r],(100756666) or (100767789),From the various human carboxylesterases only ...,NaN
1,X,NaN,NaN,EGMESTr,ecgonine methyl esterase (ER),ALKALOID SYNTHESIS,h2o[r] + egme[r] -> h[r] + ecgon[r] + meoh[r],(100771815),Encoded enzyme (100771815) is an arylacetamide...,NaN
2,X,NaN,NaN,NMPTRCOX,N-Methylputrescine:oxygen oxidoreductase (deam...,ALKALOID SYNTHESIS,nmptrc[c] + o2[c] -> nh4[c] + 1mpyr[c] + h2o2...,(100771382) or (100762635) or (100762926) or (...,All encode amine oxidases: AOC1 (100771382); A...,NaN
3,X,NaN,NaN,PRO1x,L-ProlineNAD+ 5-oxidoreductase,ARGININE AND PROLINE METABOLISM,pro_L[c] + nad[c] -> 2 h[c] + 1pyr5c[c] + nad...,(100773901),"100773901 maps to 58510, ""similar to proline d...",NaN
4,X,NaN,NaN,DHDDH,Dihydrodiol dehydrogenase,CYP METABOLISM,nadp[c] + dhnpthld[c] -> npthld[c] + nadph[c],(100753544),Dimeric dihydrodiol dehydrogenase was found to...,NaN
...,...,...,...,...,...,...,...,...,...,...
3224,X,NaN,NaN,C8CRNt,C8:0 acylcarnitine transport to mitochondria,CARNITINE SHUTTLE,c8crn[c] -> c8crn[m],(100765000),NaN,NaN
3225,X,NaN,NaN,MALCRNt,Malonyl carnitine transport into mitochondria,CARNITINE SHUTTLE,HC10859[c] -> HC10859[m],(100765000),NaN,NaN
3226,X,NaN,NaN,NP1,nucleotide phosphatase,NAD METABOLISM,h[c] + nac[c] + r1p[c] -> pi[c] + nicrns[c],(100757729),Homolog to the human gene in Recon1,NaN
3227,X,NaN,NaN,UGT1A10r,"UDP-glucuronosyltransferase 1-10 precursor, mi...",STEROID METABOLISM,2 h[r] + udpglcur[r] + bilirub[r] <=> udp[r] ...,(100755423),PMID:7945246 shows mono and di gluconidation o...,NaN


In [4]:
cols = hefzi_df.columns.to_list()+fouladiha_df.columns.to_list()+yeo_df.columns.to_list()

# Eliminate repetitive values in the 'cols' list
cols = [cols[i] for i in range(len(cols)) if i == cols.index(cols[i])]

cols

['hef',
 'fou',
 'yeo',
 'Reaction',
 'Reaction Name',
 'Subsystem',
 'Reaction Formula',
 'GPR',
 'Curation Notes',
 'References',
 'Genes',
 'Protein',
 'Reversible',
 'Lower bound',
 'Upper bound',
 'Objective',
 'Proteins',
 'EC Number',
 'Mol wt',
 'kcat_forward',
 'kcat_backward',
 'Subsystem (iCHO2291)']

In [5]:
def add_col(df):
    '''
    This function adds the remaining columns from the cols list 
    to an existing df that are not contained in such df
    '''
    df.columns
    add_col = []
    for col in cols:
        if col not in df.columns:
            add_col.append(col)
    df = df.reindex(columns = df.columns.tolist() + add_col)
    return df

In [6]:
# Unify columns for all datasets
hefzi_df = add_col(hefzi_df)
fouladiha_df = add_col(fouladiha_df)
yeo_df = add_col(yeo_df)

hefzi_df

,hef,fou,yeo,Reaction,Reaction Name,Subsystem,Reaction Formula,GPR,Curation Notes,References,...,Reversible,Lower bound,Upper bound,Objective,Proteins,EC Number,Mol wt,kcat_forward,kcat_backward,Subsystem (iCHO2291)
0,X,NaN,NaN,COKECBESr,Carboxylesterase (cocaine) (er),ALKALOID SYNTHESIS,coke[r] + h2o[r] -> bz[r] + egme[r] + h[r],(100756666) or (100767789),From the various human carboxylesterases only ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,X,NaN,NaN,EGMESTr,ecgonine methyl esterase (ER),ALKALOID SYNTHESIS,h2o[r] + egme[r] -> h[r] + ecgon[r] + meoh[r],(100771815),Encoded enzyme (100771815) is an arylacetamide...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,X,NaN,NaN,NMPTRCOX,N-Methylputrescine:oxygen oxidoreductase (deam...,ALKALOID SYNTHESIS,nmptrc[c] + o2[c] -> nh4[c] + 1mpyr[c] + h2o2...,(100771382) or (100762635) or (100762926) or (...,All encode amine oxidases: AOC1 (100771382); A...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,X,NaN,NaN,PRO1x,L-ProlineNAD+ 5-oxidoreductase,ARGININE AND PROLINE METABOLISM,pro_L[c] + nad[c] -> 2 h[c] + 1pyr5c[c] + nad...,(100773901),"100773901 maps to 58510, ""similar to proline d...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,X,NaN,NaN,DHDDH,Dihydrodiol dehydrogenase,CYP METABOLISM,nadp[c] + dhnpthld[c] -> npthld[c] + nadph[c],(100753544),Dimeric dihydrodiol dehydrogenase was found to...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3224,X,NaN,NaN,C8CRNt,C8:0 acylcarnitine transport to mitochondria,CARNITINE SHUTTLE,c8crn[c] -> c8crn[m],(100765000),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3225,X,NaN,NaN,MALCRNt,Malonyl carnitine transport into mitochondria,CARNITINE SHUTTLE,HC10859[c] -> HC10859[m],(100765000),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3226,X,NaN,NaN,NP1,nucleotide phosphatase,NAD METABOLISM,h[c] + nac[c] + r1p[c] -> pi[c] + nicrns[c],(100757729),Homolog to the human gene in Recon1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3227,X,NaN,NaN,UGT1A10r,"UDP-glucuronosyltransferase 1-10 precursor, mi...",STEROID METABOLISM,2 h[r] + udpglcur[r] + bilirub[r] <=> udp[r] ...,(100755423),PMID:7945246 shows mono and di gluconidation o...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Reorder columns in all datasets the same way
fouladiha_df = fouladiha_df[['hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References']]
yeo_df = yeo_df[['hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References']]
hefzi_df = hefzi_df[['hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References']]

hefzi_df

In [ ]:
# Merge all the df into a unified df
all_dfs = pd.concat([hefzi_df, fouladiha_df, yeo_df])
all_dfs = all_dfs.reset_index(drop = True)

all_dfs #16901 rows/reactions (many of them repeated)

In [ ]:
# Group the entire dataset by reaction name 
all_dfs1 = all_dfs.groupby(['Reaction'], group_keys=True).apply(lambda x: x)

# Save merged datasets as Excel file
all_dfs1.to_excel('../Data/all_dfs1.xlsx')

all_dfs1

In [ ]:
#def f(x):
#    d = {}
#    d['EC Number_m'] = x['EC Number'].first()
#    d['a_max'] = x['a'].max()
#    d['b_mean'] = x['b'].mean()
#    d['c_d_prodsum'] = (x['c'] * x['d']).sum()
#    return pd.Series(d, index=['EC Number_m', 'a_max', 'b_mean', 'c_d_prodsum'])

all_dfs2 = all_dfs.groupby('Reaction').first()
all_dfs2.to_excel('../Data/all_dfs2.xlsx')
all_dfs2

In [ ]:
for r in all_dfs2.index:
    print(r)

In [ ]:
# this function for the getting information from the BiGG database

def get_rxninfo(rxn):

    #download the page
    session = HTMLSession()
    
    response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn)
    if response.status_code != 200:
        print('Status code:', response.status_code,f'Failed to fetch info on {rxn} from iCHOv1 model')
        response=session.get('http://bigg.ucsd.edu/universal/reactions/'+rxn)
        if response.status_code != 200:
            print('Status code:', response.status_code,f'Failed to fetch info on {rxn} from BiGG')

    #parse using beautiful soup
    rxn_doc = BeautifulSoup(response.text,'html.parser')

    return rxn_doc


def bigg_attributes(rxn):
    
    '''
    recieves a rxn file from the get_rxninfo function
    and returns Reaction Description, Subsystem
    '''
    
    # Reaction description
    rxn_d = rxn.find_all("p")[0].get_text()
    
    # Reaction formula
    form = rxn.find_all("p")[2].get_text()
    
    # Subsystem
    subsystem = rxn.find_all("p")[5].get_text()
    
    return rxn_d, form, subsystem


In [ ]:
#all_dfs3 = pd.DataFrame(columns = ['Reactions', 'Reaction Description', 'Reaction Formula', 'Subsystem'])
att = []
for r in all_dfs2.index:
    try:
        rxn = get_rxninfo(r)
        rxn_d, form, subsystem = bigg_attributes(rxn)
        print(rxn_d, form, subsystem)
    except:
        continue
    

In [ ]:
rxn.find_all("p")

In [ ]:
rxn.find_all("p")[2].get_text()

In [ ]:
rxn.find_all("div", class_="col-lg-8")